In [44]:
library(LiblineaR)
library(pROC)
library(useful)
library(caret)
library(ROCR)
library(clue)
library(fpc)
library(clues)
library(pdist)
library(tidyr)

In [6]:
alldata= read.csv("Frogs_MFCCs.csv")
alldata$Family <- as.factor(alldata$Family)
alldata$Genus <- as.factor(alldata$Genus)
alldata$Species <- as.factor(alldata$Species)

In [7]:
dim(alldata)

[1] 7195   26

In [39]:
#CH to find optimal k
for (i in 7:20){
    km = kmeans(alldata[1:22], i)
#     print(names(km))
    B=km$betweenss
    W=km$tot.withinss
    CH = B*(nrow(alldata[1:22])-i)/((i-1)*W)
    print(i)
    print(CH)
}

[1] 7
[1] 1758.525
[1] 8
[1] 1936.112
[1] 9
[1] 1907.195
[1] 10
[1] 1888.083
[1] 11
[1] 1794.541
[1] 12
[1] 1690.3
[1] 13
[1] 1575.92
[1] 14
[1] 1555.312
[1] 15
[1] 1473.464
[1] 16
[1] 1566.121
[1] 17
[1] 1283.872
[1] 18
[1] 1433.734
[1] 19
[1] 1429.261
[1] 20
[1] 1258.767


In [40]:
optimalk = 8

In [46]:
#do optimal kmeans
km = kmeans(alldata[1:22],optimalk, nstart= 25)


centers=km$centers

In [78]:
#family

for (i in 1:8){
    clusteri=data.frame()
    tmpdata = alldata
    tmpdata$dist = apply(tmpdata,1,function(x) sqrt(sum((as.numeric(x[1:22])-centers[i,])^2)))

    for (j in 1:30){
        tmpmin =which(tmpdata$dist==(min(tmpdata$dist)))
        if (length(tmpmin)>=1){
            tmpmin = tmpmin[1]
        }
        clusteri = rbind(clusteri, tmpdata[tmpmin,])
        tmpdata = tmpdata[-tmpmin,]


    }
    print(i)
    print(summary(clusteri$Family))
}

[1] 1
      Bufonidae   Dendrobatidae         Hylidae Leptodactylidae 
              0               0               0              30 
[1] 2
      Bufonidae   Dendrobatidae         Hylidae Leptodactylidae 
              0              27               1               2 
[1] 3
      Bufonidae   Dendrobatidae         Hylidae Leptodactylidae 
              0               0              30               0 
[1] 4
      Bufonidae   Dendrobatidae         Hylidae Leptodactylidae 
              0               0              30               0 
[1] 5
      Bufonidae   Dendrobatidae         Hylidae Leptodactylidae 
              0               0               0              30 
[1] 6
      Bufonidae   Dendrobatidae         Hylidae Leptodactylidae 
              0               0               0              30 
[1] 7
      Bufonidae   Dendrobatidae         Hylidae Leptodactylidae 
              0               0              30               0 
[1] 8
      Bufonidae   Dendrobatidae         Hy

In [79]:
#genus
for (i in 1:8){
    clusteri=data.frame()
    tmpdata = alldata
    tmpdata$dist = apply(tmpdata,1,function(x) sqrt(sum((as.numeric(x[1:22])-centers[i,])^2)))

    for (j in 1:30){
        tmpmin =which(tmpdata$dist==(min(tmpdata$dist)))
        if (length(tmpmin)>=1){
            tmpmin = tmpmin[1]
        }
        clusteri = rbind(clusteri, tmpdata[tmpmin,])
        tmpdata = tmpdata[-tmpmin,]


    }
    print(i)
    print(summary(clusteri$Genus))
}

[1] 1
    Adenomera      Ameerega Dendropsophus     Hypsiboas Leptodactylus 
           30             0             0             0             0 
Osteocephalus      Rhinella        Scinax 
            0             0             0 
[1] 2
    Adenomera      Ameerega Dendropsophus     Hypsiboas Leptodactylus 
            2            27             1             0             0 
Osteocephalus      Rhinella        Scinax 
            0             0             0 
[1] 3
    Adenomera      Ameerega Dendropsophus     Hypsiboas Leptodactylus 
            0             0             0            30             0 
Osteocephalus      Rhinella        Scinax 
            0             0             0 
[1] 4
    Adenomera      Ameerega Dendropsophus     Hypsiboas Leptodactylus 
            0             0             0            30             0 
Osteocephalus      Rhinella        Scinax 
            0             0             0 
[1] 5
    Adenomera      Ameerega Dendropsophus     Hypsiboas Le

In [92]:
#species
for (i in 1:8){
    clusteri=data.frame()
    tmpdata = alldata
    tmpdata$dist = apply(tmpdata,1,function(x) sqrt(sum((as.numeric(x[1:22])-centers[i,])^2)))

    for (j in 1:30){
        tmpmin =which(tmpdata$dist==(min(tmpdata$dist)))
        if (length(tmpmin)>=1){
            tmpmin = tmpmin[1]
        }
        clusteri = rbind(clusteri, tmpdata[tmpmin,])
        tmpdata = tmpdata[-tmpmin,]


    }
    print(i)
    print(summary(clusteri$Species))
}

[1] 1
        AdenomeraAndre AdenomeraHylaedactylus     Ameeregatrivittata 
                    30                      0                      0 
            HylaMinuta   HypsiboasCinerascens      HypsiboasCordobae 
                     0                      0                      0 
   LeptodactylusFuscus  OsteocephalusOophagus      Rhinellagranulosa 
                     0                      0                      0 
           ScinaxRuber 
                     0 
[1] 2
        AdenomeraAndre AdenomeraHylaedactylus     Ameeregatrivittata 
                     2                      0                     27 
            HylaMinuta   HypsiboasCinerascens      HypsiboasCordobae 
                     1                      0                      0 
   LeptodactylusFuscus  OsteocephalusOophagus      Rhinellagranulosa 
                     0                      0                      0 
           ScinaxRuber 
                     0 
[1] 3
        AdenomeraAndre AdenomeraHylaedactylus 

In [112]:
#label custered data by polling results
preddata=alldata


for (i in 1:nrow(alldata)){
    if (km$cluster[i]==1){
        preddata[i,]$Family="Leptodactylidae"
        preddata[i,]$Genus="Adenomera"
        preddata[i,]$Species="AdenomeraAndre"
    }
    else if (km$cluster[i]==2){
        preddata[i,]$Family="Dendrobatidae"
        preddata[i,]$Genus="Ameerega"
        preddata[i,]$Species="Ameeregatrivittata"
    }
    else if (km$cluster[i]==3){
        preddata[i,]$Family="Hylidae"
        preddata[i,]$Genus="Hypsiboas"
        preddata[i,]$Species="HypsiboasCinerascens"
    }
    else if (km$cluster[i]==4){
        preddata[i,]$Family="Hylidae"
        preddata[i,]$Genus="Hypsiboas"
        preddata[i,]$Species="HypsiboasCordobae"
    }
    else if (km$cluster[i]==5){
        preddata[i,]$Family="Leptodactylidae"
        preddata[i,]$Genus="Adenomera"
        preddata[i,]$Species="AdenomeraHylaedactylus"
    }
    else if (km$cluster[i]==6){
        preddata[i,]$Family="Leptodactylidae"
        preddata[i,]$Genus="Adenomera"
        preddata[i,]$Species="AdenomeraHylaedactylus"
    }
    else if (km$cluster[i]==7){
        preddata[i,]$Family="Hylidae"
        preddata[i,]$Genus="Hypsiboas"
        preddata[i,]$Species="HypsiboasCordobae"
    }
    else if (km$cluster[i]==8){
        preddata[i,]$Family="Leptodactylidae"
        preddata[i,]$Genus="Adenomera"
        preddata[i,]$Species="AdenomeraHylaedactylus"
    }
}

In [123]:
#calculating average hamming distance
hd = 0
for (i in 1:nrow(alldata)){
    if (preddata[i,]$Family!=alldata[i,]$Family){
        hd = hd+1
    }
    if (preddata[i,]$Genus!=alldata[i,]$Genus){
        hd = hd+1
    }
    if (preddata[i,]$Species!=alldata[i,]$Species){
        hd = hd+1
    }
#     print(i)
}
avghm=hd/(nrow(alldata)*3)

In [121]:
avghm

[1] 0.1989344